In [1]:
import os
os.listdir()


['.ipynb_checkpoints',
 'analysis.ipynb',
 'orders.csv',
 'restaurants.sql',
 'users.json']

In [2]:
import pandas as pd
import json
import sqlite3


In [3]:
orders = pd.read_csv("orders.csv")

In [4]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [6]:
with open("users.json", "r") as f:
    users_data = json.load(f)

In [7]:
users = pd.DataFrame(users_data)

In [8]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [9]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [10]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [11]:
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

In [12]:
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

In [13]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [14]:
merged_df = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

In [15]:
final_df = pd.merge(
    merged_df,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [16]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [18]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [19]:
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [20]:
user_spending = final_df.groupby('user_id')['total_amount'].sum()

(user_spending > 1000).sum()

2544

In [21]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)


In [22]:
final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [23]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [24]:
restaurant_count = final_df.groupby('cuisine')['restaurant_id'] \
    .nunique() \
    .reset_index(name='restaurants')

revenue = final_df.groupby('cuisine')['total_amount'] \
    .sum() \
    .reset_index(name='revenue')

cuisine_analysis = pd.merge(
    restaurant_count,
    revenue,
    on='cuisine'
)

cuisine_analysis.sort_values(
    by=['restaurants', 'revenue'],
    ascending=[True, False]
)

,cuisine,restaurants,revenue
0,Chinese,120,1930504.65
2,Italian,126,2024203.80
1,Indian,126,1971412.58
3,Mexican,128,2085503.09


In [25]:
gold_orders = final_df[final_df['membership'] == 'Gold'].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)

50

In [26]:

restaurant_stats = (
    final_df
    .groupby('restaurant_name_x', as_index=False)
    .agg(
        avg_order_value=('total_amount', 'mean'),
        order_count=('order_id', 'count')
    )
)

less than 20 orders
filtered = restaurant_stats.query("order_count < 20")


filtered.sort_values(
    by='avg_order_value',
    ascending=False
).head(1)

,restaurant_name_x,avg_order_value,order_count
173,Hotel Dhaba Multicuisine,1040.222308,13


In [27]:
final_df[['restaurant_name_x', 'restaurant_name_y']].drop_duplicates().head(10)

,restaurant_name_x,restaurant_name_y
0,New Foods Chinese,Restaurant_450
1,Ruchi Curry House Multicuisine,Restaurant_309
2,Spice Kitchen Punjabi,Restaurant_107
3,Darbar Kitchen Non-Veg,Restaurant_224
4,Royal Eatery South Indian,Restaurant_293
5,Annapurna Tiffins South Indian,Restaurant_499
6,Royal Biryani North Indian,Restaurant_35
7,Spice Mess Punjabi,Restaurant_57
8,Ruchi Biryani Punjabi,Restaurant_7
9,Taste of Biryani Non-Veg,Restaurant_183


In [28]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered[filtered['restaurant_name_x'].isin(options)] \
    .sort_values('avg_order_value', ascending=False)

,restaurant_name_x,avg_order_value,order_count
254,Ruchi Foods Chinese,686.603158,19


In [29]:
# Step 1: Define only the MCQ options
valid_combinations = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

# Step 2: Filter to those combinations only
filtered_combos = (
    final_df
    .groupby(['membership', 'cuisine'], as_index=False)
    .agg(total_revenue=('total_amount', 'sum'))
)

filtered_combos = filtered_combos[
    filtered_combos[['membership', 'cuisine']]
    .apply(tuple, axis=1)
    .isin(valid_combinations)
]

# Step 3: Sort to find highest revenue among options
filtered_combos.sort_values('total_revenue', ascending=False)


,membership,cuisine,total_revenue
2,Gold,Italian,1005779.05
5,Regular,Indian,992100.27
1,Gold,Indian,979312.31
4,Regular,Chinese,952790.91


In [30]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_22848\606004383.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


In [31]:
final_df['quarter'] = final_df['order_date'].dt.quarter

In [32]:
quarter_revenue = (
    final_df
    .groupby('quarter', as_index=False)
    .agg(total_revenue=('total_amount', 'sum'))
    .sort_values('total_revenue', ascending=False)
)

quarter_revenue

,quarter,total_revenue
2,3,2037385.10
3,4,2018263.66
0,1,2010626.64
1,2,1945348.72


In [33]:
gold_orders_count = final_df[final_df['membership'] == 'Gold'].shape[0]
gold_orders_count

4987

In [34]:
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()

round(hyderabad_revenue)

1889367

In [35]:
distinct_users = final_df['user_id'].nunique()
distinct_users


2883

In [36]:
gold_avg_order_value = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()

round(gold_avg_order_value, 2)

797.15

In [37]:
high_rating_orders = final_df[final_df['rating'] >= 4.5].shape[0]
high_rating_orders


3374

In [38]:
gold_city_revenue = (
    final_df[final_df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [39]:
top_gold_city = gold_city_revenue.index[0]

gold_orders_top_city = final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == top_gold_city)
].shape[0]

gold_orders_top_city

1337